## Задание

1. Мы будем работать с (частичными) данными lenta.ru отсюда: https://www.kaggle.com/yutkin/corpus-of-russian-news-articles-from-lenta/
2. Проведите препроцессинг текста. Разбейте данные на train и test для задачи классификации (в качестве метки класса будем использовать поле topic). В качестве данных для классификации в пунктах 3 и 5 возьмите
    - только заголовки (title)
    - только тексты новости (text)
    - и то, и другое
3. Обучите fastText для классификации текстов по темам. Сравните качество для разных данных из п. 2.
4. Обучите свою модель w2v (или возьмите любую подходящую предобученную модель). Реализуйте функцию для вычисления вектора текста / заголовка / текста+заголовка как среднего вектора входящих в него слов. 
     - (Бонус) Модифицируйте функцию вычисления среднего вектора: взвешивайте вектора слов соответствующими весами tf-idf.
5. Обучите на полученных средних векторах алгоритм классификации, сравните полученное качество с классификатором fastText. 

In [1]:
import os

!pip install pymystem3 -q
!pip install fasttext -q
!pip install gensim -q
!pip install nltk -q

import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt

from pymystem3 import Mystem

import nltk
from nltk.corpus import stopwords

from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score

import fasttext

import urllib.request

import gensim
from gensim.models import Word2Vec

from sklearn.linear_model import LogisticRegression

from sklearn.feature_extraction.text import TfidfVectorizer

### Загрузка датасета

In [2]:
# !wget -O data/lenta-ru-news-part.csv https://www.dropbox.com/s/ja23c9l1ppo9ix7/lenta-ru-news-part.csv?dl=0

In [3]:
# !kaggle datasets download -d yutkin/corpus-of-russian-news-articles-from-lenta

In [4]:
data_dir = './data'

In [5]:
# !unzip data/archive.zip -d data/

In [6]:
lenta = pd.read_csv('data/lenta-ru-news.csv', usecols=['title', 'text', 'topic'], low_memory=False)
lenta.head()

,title,text,topic
0,1914. Русские войска вступили в пределы Венгрии,Бои у Сопоцкина и Друскеник закончились отступ...,Библиотека
1,1914. Празднование столетия М.Ю. Лермонтова от...,"Министерство народного просвещения, в виду про...",Библиотека
2,1914. Das ist Nesteroff!,"Штабс-капитан П. Н. Нестеров на днях, увидев в...",Библиотека
3,1914. Бульдог-гонец под Льежем,Фотограф-корреспондент Daily Mirror рассказыва...,Библиотека
4,1914. Под Люблином пойман швабский зверь,"Лица, приехавшие в Варшаву из Люблина, передаю...",Библиотека


#### Оставим только классы которые подразумевались в задании

In [7]:
target_topics = [
    'Экономика',
    'Спорт',
    'Культура',
    'Наука и техника',
    'Бизнес'
] 

lenta = lenta[lenta['topic'].isin(target_topics)]

In [8]:
lenta.topic.value_counts()

topic
Экономика          79528
Спорт              64413
Культура           53797
Наука и техника    53136
Бизнес              7399
Name: count, dtype: int64

### Препроцессинг

In [9]:
mystem_analyzer = Mystem()

In [10]:
russian_stopwords = stopwords.words('russian')

In [11]:
def lemmatize(text):
    tokens = mystem_analyzer.lemmatize(text)
    word_tokens = [token for token in tokens if token.isalpha()]
    return word_tokens

def clean_stopwords(tokens):
    non_stopword_tokens = [token for token in tokens if token not in russian_stopwords]
    return non_stopword_tokens

def preprocess_text(text):
    lemma_tokens = lemmatize(text.lower())
    clean_tokens = clean_stopwords(lemma_tokens)
    return ' '.join(clean_tokens)

In [12]:
tqdm.pandas()

In [13]:
lenta = lenta[lenta.title.apply(lambda x: isinstance(x, str))]

lenta = lenta[lenta.text.apply(lambda x: isinstance(x, str))]

lenta = lenta[lenta.title.notna()]

lenta = lenta[lenta.text.notna()]

In [14]:
lenta_lemmatized_path = os.path.join(data_dir, 'lenta_lemmatized.csv')

if not os.path.exists(lenta_lemmatized_path):
    lenta['title_lemma'] = lenta['title'].progress_apply(preprocess_text)

    lenta['text_lemma'] = lenta['text'].progress_apply(preprocess_text)

    lenta.to_csv(os.path.join(data_dir, 'lenta_lemmatized.csv'), index=False)
else:
    print("lenta_lemmatized is already exists")

lenta_lemmatized is already exists


In [15]:
lenta_lemmatized = pd.read_csv(os.path.join(data_dir, 'lenta_lemmatized.csv'), index_col=0)

In [16]:
lenta_lemmatized = lenta_lemmatized[
    lenta_lemmatized['text_lemma'].apply(lambda x: isinstance(x, str)) & 
    lenta_lemmatized['title_lemma'].apply(lambda x: isinstance(x, str))
]

In [17]:
assert lenta_lemmatized['text_lemma'].apply(lambda x: isinstance(x, str)).all()

In [18]:
assert lenta_lemmatized['title_lemma'].apply(lambda x: isinstance(x, str)).all()

In [19]:
lenta_lemmatized.head()

,text,topic,title_lemma,text_lemma
title,,,,
Телеканалы станут вещать по единому тарифу,С 1 января 2000 года все телеканалы будут опла...,Экономика,телеканал становиться вещать единый тариф,январь год весь телеканал оплачивать услуга пе...
"Volkswagen выкупает остатки акций ""Шкоды""",Германский автопромышленный концерн Volkswagen...,Экономика,volkswagen выкупать остаток акция шкода,германский автопромышленный концерн volkswagen...
Прибыль Тюменнефтегаза возросла в 10 раз,"Нераспределенная прибыль ОАО ""Тюменнефтегаз"", ...",Экономика,прибыль тюменнефтегаз возрастать,нераспределенный прибыль оао тюменнефтегаз доч...
Крупнейшее в истории слияние компаний происходит в США,Две крупнейших телекоммуникационных компании С...,Экономика,крупный история слияние компания происходить сша,крупный телекоммуникационный компания сша дост...
ГАЗ получил четверть обещанного кредита,"ОАО ""ГАЗ"" и Нижегородский банк Сбербанка Росси...",Экономика,газ получать четверть обещать кредит,оао газ нижегородский банк сбербанк россия под...


### Train-Test

In [20]:
train_lenta, test_lenta = train_test_split(
    lenta_lemmatized,
    train_size=0.7,
    stratify=lenta_lemmatized['topic'],
    shuffle=True,
    random_state=667
)

### FastText

In [21]:
# Фасттекст хавает данные только в таком странном виде
def write2txt(text_list, label_list, filename):
    save_path = os.path.join(data_dir, filename + '.txt')
    with open(save_path, 'w') as file:
        for sentence, label in zip(text_list, label_list):
            file.write(f"__label__{label} {sentence}")
            file.write('\n')

    return save_path

In [22]:
def compute_metrics(
    true_labels,
    predicted_labels
):
    score_accuracy = accuracy_score(true_labels, predicted_labels)
    micro_f1 = f1_score(true_labels, predicted_labels, average='micro')
    macro_f1 = f1_score(true_labels, predicted_labels, average='macro')
    weighted_f1 = f1_score(true_labels, predicted_labels, average='weighted')
    
    metric_entry = {
        'accuracy' : score_accuracy,
        'micro_f1' : micro_f1,
        'macro_f1' : macro_f1,
        'weighted_f1' : weighted_f1
    }
    return metric_entry

def train_and_evaluate(
    train_sentences, 
    train_labels,
    test_sentences,
    test_labels,
    filename
):
    assert len(train_sentences) == len(train_labels)
    assert len(test_sentences) == len(test_labels)
    
    save_path = write2txt(
        train_sentences,
        train_labels,
        filename
    )
    
    model = fasttext.train_supervised(save_path, wordNgrams=2)

    delete_label_prefix = lambda word: word[len('__label__'):]
    
    predicted_labels_with_prefix = model.predict(test_sentences)[0]
    predicted_labels = [delete_label_prefix(label[0]) for label in predicted_labels_with_prefix]

    metric_entry = compute_metrics(test_labels, predicted_labels)
    
    for name, value in metric_entry.items():
        print(f"{name} : {round(value, 3)}")

    return metric_entry

In [23]:
# только на titles
title_metrics = train_and_evaluate(
    train_lenta['title_lemma'].tolist(),
    train_lenta['topic'].tolist(),
    test_lenta['title_lemma'].tolist(),
    test_lenta['topic'].tolist(),
    'title_lenta'
)

Read 1M words
Number of words:  43035
Number of labels: 5
Progress: 100.0% words/sec/thread:  613506 lr:  0.000000 avg.loss:  0.089309 ETA:   0h 0m 0s


accuracy : 0.753
micro_f1 : 0.753
macro_f1 : 0.498
weighted_f1 : 0.669


In [24]:
# только на full text
text_metrics = train_and_evaluate(
    train_lenta['text_lemma'].tolist(),
    train_lenta['topic'].tolist(),
    test_lenta['text_lemma'].tolist(),
    test_lenta['topic'].tolist(),
    'text_lemma'
)

Read 23M words
Number of words:  250947
Number of labels: 5
Progress: 100.0% words/sec/thread: 1097572 lr:  0.000000 avg.loss:  0.109028 ETA:   0h 0m 0s


accuracy : 0.771
micro_f1 : 0.771
macro_f1 : 0.572
weighted_f1 : 0.737


In [25]:
def join_series(lhs, rhs):
    return lhs.str.cat(rhs, sep=" ")

In [26]:
# Concatenated title and text
title_text_metrics = train_and_evaluate(
    join_series(train_lenta['title_lemma'], train_lenta['text_lemma']).tolist(),
    train_lenta['topic'].tolist(),
    join_series(test_lenta['title_lemma'], test_lenta['text_lemma']).tolist(),
    test_lenta['topic'].tolist(),
    'text_lemma'
)

Read 25M words
Number of words:  252060
Number of labels: 5
Progress: 100.0% words/sec/thread: 1098749 lr:  0.000000 avg.loss:  0.110103 ETA:   0h 0m 0s 16.6% words/sec/thread: 1114218 lr:  0.083410 avg.loss:  0.356516 ETA:   0h 0m 3s


accuracy : 0.772
micro_f1 : 0.772
macro_f1 : 0.576
weighted_f1 : 0.74


### Word2Vec

In [28]:
def vectorize_word_list(w2v_model, word_list):
    vector_size = w2v_model.wv.vector_size
    vectors = np.array([w2v_model.wv.get_vector(word) for word in word_list if word in w2v_model.wv])
    if len(vectors) == 0:
        vectors = np.array([[0 for index in range(vector_size)]])
    mean_vector = vectors.mean(axis=0)
    assert mean_vector.shape[0] == vector_size
    return mean_vector

def vectorize(w2v_model, sentences):
    vectors = [vectorize_word_list(w2v_model, token_sentence) for token_sentence in sentences]
    return vectors

In [29]:
def pipeline(
    train_text,
    train_label,
    test_text,
    test_label,
    model_name='word2vec'
):
    whitespace_tokenizer = nltk.WhitespaceTokenizer()
    train_tokens = whitespace_tokenizer.tokenize_sents(train_text)
    test_tokens = whitespace_tokenizer.tokenize_sents(test_text)

    model_path = f'data/{model_name}.model'
    if not os.path.exists(model_path):
        vectorizer_model = Word2Vec(
            train_tokens,
            workers=4,
            vector_size=300,
            min_count=1,
            window=5,
            sg=1,
            sample=1e-3
        )
        vectorizer_model.save(model_path)
    else:
        print(f'{model_path} already exists. Just loading it')
    vectorizer_model = gensim.models.Word2Vec.load(model_path)

    train_vectors = vectorize(vectorizer_model, train_tokens)
    test_vectors = vectorize(vectorizer_model, test_tokens)
    
    logreg_model = LogisticRegression(
        max_iter=1000,
        random_state=667
    )
    logreg_model.fit(train_vectors, train_label)

    test_predictions = logreg_model.predict(test_vectors)
    metrics = compute_metrics(test_label, test_predictions)

    for name, value in metrics.items():
        print(f"{name} : {round(value, 3)}")

In [30]:
# Only on titles
pipeline(
    train_lenta['title_lemma'].tolist(),
    train_lenta['topic'].tolist(),
    test_lenta['title_lemma'].tolist(),
    test_lenta['topic'].tolist(),
    model_name='word2vec_title'
)

accuracy : 0.896
micro_f1 : 0.896
macro_f1 : 0.742
weighted_f1 : 0.886


In [31]:
# Only on full text
pipeline(
    train_lenta['text_lemma'].tolist(),
    train_lenta['topic'].tolist(),
    test_lenta['text_lemma'].tolist(),
    test_lenta['topic'].tolist(),
    model_name='word2vec_text'
)

accuracy : 0.955
micro_f1 : 0.955
macro_f1 : 0.852
weighted_f1 : 0.95


In [32]:
# Concatenated title and full text
pipeline(
    join_series(train_lenta['title_lemma'], train_lenta['text_lemma']).tolist(),
    train_lenta['topic'].tolist(),
    join_series(test_lenta['title_lemma'], test_lenta['text_lemma']).tolist(),
    test_lenta['topic'].tolist(),
    model_name='word2vec_title_concat_text'
)

accuracy : 0.955
micro_f1 : 0.955
macro_f1 : 0.853
weighted_f1 : 0.951


Получил довольно хорошие скоры, странно что в прошлый раз получал около ~0.7

Макро-ф1 просел так как классификация хуже на одном минорном классе

Теперь попробую взвесить вектор весами tf-idf

### TF-IDF Weight Vectorization

In [59]:
def vectorize_word_list_tfidf(w2v_model, idf_values, word_list):
    vector_size = w2v_model.wv.vector_size
    get_idf_weight = lambda word: idf_values[word] if word in idf_values else 1
    
    vectors = np.array([get_idf_weight(word) * w2v_model.wv.get_vector(word) for word in word_list if word in w2v_model.wv])
    if len(vectors) == 0:
        vectors = np.array([[0 for index in range(vector_size)]])
    mean_vector = vectors.mean(axis=0)
    assert mean_vector.shape[0] == vector_size
    return mean_vector

def vectorize_tfidf(w2v_model, idf_values, sentences):
    vectors = [vectorize_word_list_tfidf(w2v_model, idf_values, token_sentence) for token_sentence in sentences]
    return vectors

In [60]:
def pipeline_tfidf(
    train_text,
    train_label,
    test_text,
    test_label,
    model_name='word2vec_tfidf'
):
    whitespace_tokenizer = nltk.WhitespaceTokenizer()
    train_tokens = whitespace_tokenizer.tokenize_sents(train_text)
    test_tokens = whitespace_tokenizer.tokenize_sents(test_text)

    model_path = f'data/{model_name}.model'
    if not os.path.exists(model_path):
        vectorizer_model = Word2Vec(
            train_tokens,
            workers=4,
            vector_size=300,
            min_count=0,
            window=5,
            sg=1,
            sample=1e-3
        )
        vectorizer_model.save(model_path)
    else:
        print(f'{model_path} already exists. Just loading it')
    vectorizer_model = gensim.models.Word2Vec.load(model_path)

    tfidf_vectorizer = TfidfVectorizer(
        tokenizer=None,
        preprocessor=None,
        analyzer="word"
    )
    tfidf_vectorizer.fit(train_text)
    idf_values = dict(zip(tfidf_vectorizer.get_feature_names_out(), tfidf_vectorizer.idf_))
    
    
    train_vectors = vectorize_tfidf(vectorizer_model, idf_values, train_tokens)
    test_vectors = vectorize_tfidf(vectorizer_model, idf_values, test_tokens)
    
    logreg_model = LogisticRegression(
        max_iter=1000,
        random_state=667
    )
    logreg_model.fit(train_vectors, train_label)

    test_predictions = logreg_model.predict(test_vectors)
    metrics = compute_metrics(test_label, test_predictions)

    for name, value in metrics.items():
        print(f"{name} : {round(value, 3)}")

In [63]:
# Only on titles
pipeline_tfidf(
    train_lenta['title_lemma'].tolist(),
    train_lenta['topic'].tolist(),
    test_lenta['title_lemma'].tolist(),
    test_lenta['topic'].tolist(),
    model_name='word2vec_title_tfidf'
)

/opt/conda/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


accuracy : 0.902
micro_f1 : 0.902
macro_f1 : 0.757
weighted_f1 : 0.893


In [64]:
# Only on full text
pipeline_tfidf(
    train_lenta['text_lemma'].tolist(),
    train_lenta['topic'].tolist(),
    test_lenta['text_lemma'].tolist(),
    test_lenta['topic'].tolist(),
    model_name='word2vec_text_tfidf'
)

accuracy : 0.955
micro_f1 : 0.955
macro_f1 : 0.862
weighted_f1 : 0.952


In [65]:
# Concatenated title and full text
pipeline_tfidf(
    join_series(train_lenta['title_lemma'], train_lenta['text_lemma']).tolist(),
    train_lenta['topic'].tolist(),
    join_series(test_lenta['title_lemma'], test_lenta['text_lemma']).tolist(),
    test_lenta['topic'].tolist(),
    model_name='word2vec_title_concat_text_tfidf'
)

accuracy : 0.956
micro_f1 : 0.956
macro_f1 : 0.861
weighted_f1 : 0.953


Tf-Idf чуть улучшил скор. На 1 процент. Мб стоит поиграться с умножением не на 1.

Размер ембеддинга 300, при тесте больших моделей на бенчмарках 300 дает очень хорошие результаты и размеры 512 и 1024 не дают сильного прироста.